In [ ]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random as rnd
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [ ]:
data  = pd.read_csv('ionosphereData.csv', header=None)

headers = ["Attribute"+str(i) for i in range(1,35)]
headers.append("Class") # Add the class label column
data.columns = headers

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
if (data["Attribute2"] == 0).all():
    print("All values in column 2 are 0.")
    print("Hence the column is redundant and is dropped.")
    data = data.drop(columns=["Attribute2"], axis=1)

else:
    print("Not all values in column 2 are 0")

In [ ]:
data.shape

In [ ]:
columns = data.columns

correlationMatrix = data.corr()

In [ ]:
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values     


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
print("X_train shape: ", X_train.shape)
print("X_test shape: ", X_test.shape)
print("y_train shape: ", y_train.shape)
print("y_test shape: ", y_test.shape)

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)

X_test = scaler.transform(X_test)


In [ ]:
# Convert 'g' to 1 and 'b' to 0
y_train = (y_train == 'g').astype(int)
y_test = (y_test == 'g').astype(int)


In [ ]:
class Layer:
    def __init__(self):
        self.input = None
        self.output = None

    def forward(self, input):
        
        pass

    def backward(self, output_gradient, learning_rate):
        
        pass

In [ ]:
class Dense(Layer):
    def __init__(self, input_size, output_size):
        self.weights = np.random.randn(output_size, input_size)
        self.bias = np.random.randn(output_size, 1)

    def forward(self, input):
        self.input = input
        return np.dot(self.weights, self.input) + self.bias

    def backward(self, output_gradient, learning_rate):
        weights_gradient = np.dot(output_gradient, self.input.T)
        input_gradient = np.dot(self.weights.T, output_gradient)
        self.weights -= learning_rate * weights_gradient
        self.bias = self.bias - learning_rate * output_gradient
        return input_gradient

In [ ]:
class Activation(Layer):
    def __init__(self, activation, activation_prime):
        self.activation = activation
        self.activation_prime = activation_prime

    def forward(self, input):
        self.input = input
        return self.activation(self.input)

    def backward(self, output_gradient, learning_rate):
        return np.multiply(output_gradient, self.activation_prime(self.input))

In [ ]:
class tanh(Activation):
    def __init__(self):
        def tanh(x):
            return np.tanh(x)

        def tanh_prime(x):
            return 1 - np.tanh(x) ** 2

        super().__init__(tanh, tanh_prime)

class sigmoid(Activation):
    def __init__(self):
        def sigmoid(x):
            return 1 / (1 + np.exp(-x))

        def sigmoid_prime(x):
            s = sigmoid(x)
            return s * (1 - s)

        super().__init__(sigmoid, sigmoid_prime)

class Relu(Activation):
    def __init__(self):

        def relu(x):
            return np.maximum(0,x)
        
        def relu_prime(x):
            return np.where(x<=0, 0, 1)
        super().__init__(relu, relu_prime)

In [ ]:
layers = [Dense(33, 33), tanh(), Dense(33, 1), sigmoid()]

epochs = 100
learning_rate = 0.5



for epoch in range(epochs):

    error = 0
    for i in range(len(X_train)):
        instance = X_train[i]
        true_label = y_train[i]
        for layer in layers:
            instance = layer.forward(instance)
        output = instance
        error += np.square(true_label-output)
        output_gradient = -2*(true_label-output)
        for layer in reversed(layers):
            output_gradient = layer.backward(output_gradient, learning_rate)
    
    #print("Epoch: ", epoch, " Error: ", error/len(X_train))